In [378]:
import requests, json
from datetime import datetime
import math
import urllib.parse

In [379]:

def get_usd_curreny():
    dt = datetime.today()
    today = dt.strftime('%Y%m%d')

    data = {"dataBody":{"ricInptRootInfo":{"serviceType":"GU","serviceCode":"F3730","nextServiceCode":"","pkcs7Data":"","signCode":"","signData":"","useSign":"","useCert":"","permitMultiTransaction":"","keepTransactionSession":"","skipErrorMsg":"","mode":"","language":"ko","exe2e":"","hideProcess":"","clearTarget":"","callBack":"shbObj.fncF3730Callback","exceptionCallback":"","requestMessage":"","responseMessage":"","serviceOption":"","pcLog":"","preInqForMulti":"","makesum":"","removeIndex":"","redirectUrl":"","preInqKey":"","_multi_transfer_":"","_multi_transfer_count_":"","_multi_transfer_amt_":"","userCallback":"","menuCode":"","certtype":"","fromMulti":"","fromMultiIdx":"","isRule":"N","webUri":"/index.jsp","gubun":"","tmpField2":""},"조회구분":"","조회일자":today,"고시회차":1,"조회일자_display":"","startPoint":"","endPoint":""},"dataHeader":{"trxCd":"RSHRC0213A01","language":"ko","subChannel":"49","channelGbn":"D0"}}
    url = 'https://bank.shinhan.com/serviceEndpoint/httpDigital'

    res = requests.post(url, data=json.dumps(data))


    currency_exchanges = res.json()['dataBody']['R_RIBF3730_1']
    return {'code': currency_exchanges[0]['통화CODE'], 'cur': currency_exchanges[0]['전신환매도환율']}
    #전신환매도환율
    #통화CODE
usd_cur = get_usd_curreny()

In [380]:
usd_cur['cur']

1333.7

In [381]:

def round_off(num, pos=2):
    t = str(num).split('.')
    t[1] = t[1][:pos]
    return float(".".join(t))

In [382]:
# usd_cur = {'code': 'USD', 'cur': '1296.40'}

card_fee = 0

def exCalc(v):
    visafee = v * 0.011;
    visafee = float(round_off(visafee));
    t = round(v * float(usd_cur['cur']))
    card_fee = t * 0.0018
    card_fee = round_off(card_fee, 0)
    krw = ((v + visafee) * float(usd_cur['cur'])) + card_fee
    return (v+visafee, int(krw), card_fee)

exCalc(4.45)

(4.49, 5998, 10.0)

In [383]:
def default_value():
    tar = 5999
    for i in range(600, 400, -1):
        usd = i/100
        usdv, krw, card_fee = exCalc(usd)
        if (krw < 6000):
            return (usd, krw, card_fee, usdv)
        

usd, krw, card_fee, usdv = default_value()
usd, krw, card_fee, usdv

(4.45, 5998, 10.0, 4.49)

In [384]:
currenys = {}
currencys_list = ["AED", "ARS", "AUD", "CAD", "CHF", "CLP", "CZK", "CNY", "DKK", "EUR", "GBP", "HKD", "HUF", "IDR", "INR", "JPY", "NPR", "NGN", "NZD", "NOK", "MXN", "MYR", "PHP", "PLN", "RUB", "SAR", "SGD", "SEK", "THB", "TRY", "TWD", "VND", "KRW"]
def get_currency_rate():
    currenys['USD'] = {'rate': usd_cur['cur'], 'currency_rate': '1.0'}
    dt = datetime.today()
    today = dt.strftime('%m/%d/%Y')
# 
# https://www.visakorea.com/cmsapi/fx/rates?amount=4.55&fee=2&utcConvertedDate=08%2F14%2F2023&exchangedate=08%2F14%2F2023&fromCurr=KRW&toCurr=USD
    for cur in currencys_list:
        # url = 'https://www.visakorea.com/travel-with-visa/exchange-rate-calculator.html?fromCurr=USD&toCurr={}&fee=0&utcConvertedDate={}&exchangedate={}&submitButton=Calculate+Exchange+Rates'.format(cur, today, today)
        url = 'https://www.visakorea.com/cmsapi/fx/rates?amount={}&fee=0&utcConvertedDate={}&exchangedate={}&fromCurr=USD&toCurr={}'.format(usd, today, today, cur)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        res = requests.get(url, headers=headers)
        if len(res.json()) == 1:
            currenys[cur] = {'rate': usd_cur['cur'], 'currency_rate': '0.0'}
            continue
        currency_rate = res.json()['originalValues']['fxRateVisa']
        currenys[cur] = {'rate': usd_cur['cur'], 'currency_rate': currency_rate if currency_rate != '' else '1.0'}
    return currenys


In [385]:
currenys = get_currency_rate()

In [ ]:
currenys

{'USD': {'rate': 1333.7, 'currency_rate': '1.0'},
 'AED': {'rate': 1333.7, 'currency_rate': '0.2723160503'},
 'ARS': {'rate': 1333.7, 'currency_rate': '0.0017093316'},
 'AUD': {'rate': 1333.7, 'currency_rate': '0.651199349'},
 'CAD': {'rate': 1333.7, 'currency_rate': '0.7444900239'},
 'CHF': {'rate': 1333.7, 'currency_rate': '1.1424642983'},
 'CLP': {'rate': 1333.7, 'currency_rate': '0.0011634671'},
 'CZK': {'rate': 1333.7, 'currency_rate': '0.0456183112'},
 'CNY': {'rate': 1333.7, 'currency_rate': '0.1378466925'},
 'DKK': {'rate': 1333.7, 'currency_rate': '0.1471863823'},
 'EUR': {'rate': 1333.7, 'currency_rate': '1.096198904'},
 'GBP': {'rate': 1333.7, 'currency_rate': '1.272098728'},
 'HKD': {'rate': 1333.7, 'currency_rate': '0.1279703876'},
 'HUF': {'rate': 1333.7, 'currency_rate': '0.0028691083'},
 'IDR': {'rate': 1333.7, 'currency_rate': '0.0'},
 'INR': {'rate': 1333.7, 'currency_rate': '0.0120675178'},
 'JPY': {'rate': 1333.7, 'currency_rate': '0.0069132319'},
 'NPR': {'rate': 1

In [ ]:
def get_info_with_currency_and_value(cur, v):
    # idx = currencys_list.index(cur) + 1
    return exCalc(round(v * float(currenys[cur]['currency_rate']), 2))

In [ ]:
exCalc(4.45)

(4.49, 5998, 10.0)

In [ ]:
a = get_info_with_currency_and_value('AED', 10)
a

(2.74, 3660, 6.0)

In [ ]:
def get_all_info_currency_with_value():
    currencys_list = ["USD", "AED", "ARS", "AUD", "CAD", "CHF", "CLP", "CZK", "CNY", "DKK", "EUR", "GBP", "HKD", "HUF", "IDR", "INR", "JPY", "NPR", "NGN", "NZD", "NOK", "MXN", "MYR", "PHP", "PLN", "RUB", "SAR", "SGD", "SEK", "THB", "TRY", "TWD", "VND", "KRW"]
    comma = [0,2,0,2,2,2,0,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0]
    usd, krw, card_fee, usdv = default_value()

    for idx, cur in enumerate(currencys_list):
        for i in range(10, 20000000, 1):
            t = i/100
            a = t * float(currenys[cur]['currency_rate']) 
            amount = 0
            if (float("{}{}".format(str(usd), 5)) <= a):                
                prev = t - 0.01
                if (comma[idx] == 0):
                    amount = round_off(prev, 0) 
                else:
                    amount = round(prev, 2)
                currenys[cur]['cur'] = cur
                currenys[cur]['amount'] = amount if cur != 'USD' else usdv
                currenys[cur]['usd'] = usdv
                currenys[cur]['krw'] = krw
                currenys[cur]['card_fees'] = card_fee
                break
    return currenys

In [ ]:
a = get_all_info_currency_with_value()
a


{'USD': {'rate': 1333.7,
  'currency_rate': '1.0',
  'cur': 'USD',
  'amount': 4.49,
  'usd': 4.49,
  'krw': 5998,
  'card_fees': 10.0},
 'AED': {'rate': 1333.7,
  'currency_rate': '0.2723160503',
  'cur': 'AED',
  'amount': 16.35,
  'usd': 4.49,
  'krw': 5998,
  'card_fees': 10.0},
 'ARS': {'rate': 1333.7,
  'currency_rate': '0.0017093316',
  'cur': 'ARS',
  'amount': 2606.0,
  'usd': 4.49,
  'krw': 5998,
  'card_fees': 10.0},
 'AUD': {'rate': 1333.7,
  'currency_rate': '0.651199349',
  'cur': 'AUD',
  'amount': 6.84,
  'usd': 4.49,
  'krw': 5998,
  'card_fees': 10.0},
 'CAD': {'rate': 1333.7,
  'currency_rate': '0.7444900239',
  'cur': 'CAD',
  'amount': 5.98,
  'usd': 4.49,
  'krw': 5998,
  'card_fees': 10.0},
 'CHF': {'rate': 1333.7,
  'currency_rate': '1.1424642983',
  'cur': 'CHF',
  'amount': 3.89,
  'usd': 4.49,
  'krw': 5998,
  'card_fees': 10.0},
 'CLP': {'rate': 1333.7,
  'currency_rate': '0.0011634671',
  'cur': 'CLP',
  'amount': 3829.0,
  'usd': 4.49,
  'krw': 5998,
  'ca

In [ ]:
a['PHP']

{'rate': 1333.7,
 'currency_rate': '0.0175882932',
 'cur': 'PHP',
 'amount': 253.29,
 'usd': 4.49,
 'krw': 5998,
 'card_fees': 10.0}